# Armani SageMaker instance 

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
bucket_name = 'cestino-s3'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


The part until now is equal to the [tutorial](https://aws.amazon.com/it/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/).

In [4]:
try:
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv", "./products.csv")
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./products.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


Now that we have loaded into model_data the CSV...

In [5]:
print(model_data) # ...and after a bit of debugging...
print(type(model_data)) # ...and compensation of the absurd leak of documentation...

               type                         color   
category                                            
donna       giacche                           blu  \
donna     pantaloni  grigio scuro, ghiaccio e blu   
donna     pantaloni                      fantasia   
donna      sneakers                  bianco e blu   
donna         abiti                    nero e blu   
donna       giacche                      fantasia   
donna       sandali                 argento e oro   
donna       sandali                          nero   
donna       sandali                nero e marrone   
donna       sandali                          nero   
donna       giacche                      blu navy   
donna     pantaloni               blu navy e nero   
donna        scarpe                          nero   
donna         abiti                          rosa   
donna        scarpe                   nude e nero   
donna        scarpe                   nude e nero   
donna         abiti                    blu e n

...we start splitting data into the data for training (in this case 70% of the total) an data for testing (the rest).

In [6]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(35, 13) (15, 13)


We should first get what [random_state](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) is.
Well, it's just a randomizer I guess.

In [9]:
print(train_data)
print(test_data)

               type                        color   
category                                           
uomo        giacche                          blu  \
uomo      pantaloni                     fantasia   
donna         abiti                   blu e nero   
uomo          abiti              nero e blu navy   
uomo      pantaloni                        beige   
donna        scarpe                  nude e nero   
uomo        giacche                        beige   
donna       sandali                          oro   
uomo          abiti                         nero   
donna        scarpe                  nude e nero   
donna       giacche                          blu   
uomo        giacche                     fantasia   
donna       sandali               nero e marrone   
donna     pantaloni                     fantasia   
uomo        giacche                     fantasia   
donna       sandali                         nero   
donna       sandali                argento e oro   
uomo        

In [11]:
train_data.to_csv(
    'train.csv',
    index=False,
    header=False
)
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(
        prefix, 'train/train.csv'
    )
).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

In [32]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(
    xgboost_container,role, instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=f's3://{bucket_name}/{prefix}/output',
    sagemaker_session=sess
)
xgb.set_hyperparameters(
    max_depth=5,eta=0.2,gamma=4,
    min_child_weight=6,subsample=0.8,
    silent=0,objective='binary:logistic',
    num_round=100
)

In [ ]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-22-12-32-35-271


2023-06-22 12:32:35 Starting - Starting the training job...
2023-06-22 12:32:51 Starting - Preparing the instances for training...